In [14]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
from fuzzywuzzy import process, fuzz

In [35]:
from szp_funcs import load_month, print_df, load_groups, path_to_docs

In [36]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    if len(t) == 1:
        return float(t[0])
    return float(t[0]+'.'+t[1])

In [6]:
month = 'may'
path_to_occup = 'C:\\Users\\PetukhovMD\\Desktop\\2022\\new май\\ДОНМ_Штатное_Расписание.xlsx'
res = load_month(month)
occup = pd.read_excel(path_to_occup)
ped, isp, aup = load_groups('input')

In [7]:
res

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may
0,5003021368,001-171-362 75,58500.00,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
1,5003021368,001-191-281 85,112184.88,Учитель,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
2,5003021368,001-244-107 65,118681.78,Учитель,0.0,Основное место работы,1.0,Работа,Работа,15.0,18.0
3,5003021368,001-263-234 77,79619.03,Воспитатель,0.0,Основное место работы,1.0,Отпуск основной,Работа,16.0,18.0
4,5003021368,001-322-594 80,37907.50,Специалист,0.0,Основное место работы,1.0,Работа,Работа,9.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
188387,9718071371,198-317-420 02,110851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
188388,9718071371,199-418-719 35,153851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
188389,9718071371,205-412-630 13,209199.25,Начальник группы,0.0,Основное место работы,1.0,Работа,Работа,13.0,18.0
188390,9718071371,206-135-584 35,27528.33,Бухгалтер,0.0,Основное место работы,1.0,NaN,NaN,6.0,11.0


In [11]:
res_occup = occup.iloc[:, [0, 3, 5]]
res_occup.columns = ['inn', 'job', 'stv']

In [12]:
res_occup

,inn,job,stv
0,7734408766,Повар,7
1,7734408766,Повар,1
2,7734408766,мойщик посуды,1
3,7734408766,мойщик посуды,2
4,7734408766,Заведующий производством,1
...,...,...,...
43745,7722069597,Заместитель директора,1
43746,7722069597,Заместитель директора,1
43747,7722069597,Заместитель директора,1
43748,7722069597,Заместитель директора,1


In [15]:
def matcher(row, look_up, name):
    ans = process.extractOne(row[name], look_up, scorer=fuzz.WRatio, score_cutoff=60)
    if ans:
        return ans[0]
    return ans

In [16]:
jobs = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='должности')

In [18]:
jobs.columns = ['pre_job', 'job_106']

In [20]:
look_up = jobs['pre_job'].to_list()

In [21]:
res_occup['pre_job'] = res_occup.apply(lambda row: matcher(row, look_up, 'job'), axis=1)

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_11168\714250574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_occup['pre_job'] = res_occup.apply(lambda row: matcher(row, look_up, 'job'), axis=1)


In [23]:
res_occup = pd.merge(res_occup, jobs, how='left')

In [26]:
res_occup = res_occup.drop(['job', 'pre_job'], axis=1)

In [28]:
inns = [7719286023,
7720086251,
7719260829,
7720258623,
7720291042,
7720258550,
7718200721,
7718997948,
7718228237,
7718084112,
7719033470,
7707090925,
7720029197,
7718197885,
7718193658,
7719217125,
7719247881,
7718901205,
7719882717,
7720016173,
7720675962,
7718084659,
7719025840,
7720008045,
7718792108,
7718083895,
7720602851,
7718259267,
7720824692,
7719825934,
7718987403,
7718309983,
7719894832,
7720807143,
7710640209,
7720584916,
7719867099,
7720596679,
7720792271,
7720307694,
7720325492,
7719418720,
7718220326,
7718106920,
7720260372,
7719866056,
7719035132,
7719259326,
7719031995,
7720257933,
7720261048,
7718191202,
7720573400,
7719106841,
7719286104,
7718261509,
7718261587,
7719285936,
7718198423,
7717043346,
7729413009,
7732120595,
7730160480,
7730027030,
7731270005,
7729415535,
7732112259,
7732111865,
7729776838,
7732111858,
7731230362,
7731484984,
7731173812,
7729783137,
7731173820,
7731185543,
7729759977,
7729775619,
7732507722,
7731174051,
7729423688,
7731099453,
7732119504,
7729780048,
7731243019,
7729530457,
7730712331,
7729784388,
7729788505,
7729400426,
7732004550,
7729402889,
7731582090,
7729433767,
7701293580,
7731173259,
7732117313,
7729421592,
7729531411,
7731482850,
7729784613,
7729410230,
7731462620,
7732111054,
7730029051,
7731479261,
7730026170,
7729423286,
7731173523,
7729410368,
7729409041,
7729173011,
7731173700,
7731435497,
7729417973,
7731263128,
7731281046,
7731281039,
7729434070,
7729402381,
7705694400,
7735043162,
7735096830,
7735062655,
7735002230,
7735601120,
7735062743,
7735093928,
7735083380,
7735589106,
7735096823,
7735030808,
7735066040,
7735103742,
7735062750,
7735099084,
7735071709,
7735128049,
7743085712,
7714805691,
7743868110,
7743917311,
7743021236,
7713070211,
7712022550,
7727191008,
7714923649,
7743085543,
7713781783,
7743020747,
7743911729,
7714196778,
7713288810,
7743935328,
7712022159,
7712019300,
7743024406,
7743022631,
7714031656,
7714890087,
7713288793,
7743026202,
7713338324,
7743018882,
7714909450,
7714926801,
7713782829,
7743023522,
7743928539,
7714048508,
7714199190,
7714079640,
7714204845,
7713039050,
7743024928,
7743024068,
7743904344,
7714283124,
7713785435,
7743084684,
7713780268,
7720830590,
7743018900,
7743619481,
7743905443,
7743939315,
7713784953,
7713286725,
7743021300,
7713283604,
7712013764,
7713794503,
7743024357,
7713781889,
7743043649,
7713401544,
7713799237,
7743940590,
7743085720,
7713385719,
7713385726,
7743937357,
7743085744,
7713229928,
7714093099,
7714081706,
7717666051,
7717025259,
7716200170,
7715894310,
7716530474,
7716199502,
7717053947,
7715303782,
7715304578,
7716202057,
7716789580,
7715303983,
7715152967,
7715303454,
7717082867,
7715310719,
7715992532,
7715975304,
7715441479,
7715979161,
7716771230,
7717024086,
7715789629,
7717023163,
7716199414,
7717082680,
7715303091,
7716199950,
7715969660,
7715616552,
7716199492,
7715982037,
7716199407,
7715731322,
7716696769,
7715446501,
7715468897,
7716209782,
7715018947,
7715969269,
7716783901,
7715446974,
7715422010,
7715037562,
7715107749,
7715446854,
9715217689,
9715206976,
7716585000,
7715106872,
7716237684,
7716081388,
7716237691,
7716079082,
7715398103,
7716201078,
7734570529,
7734687492,
7734052005,
7734077088,
7734626387,
7733132610,
7734233770,
7733127762,
7733181390,
7734248978,
7733113047,
7733088746,
7733558487,
7734247614,
7733834994,
7734686844,
7733672292,
7733114266,
7734229661,
7734132109,
7734241813,
7734236901,
7733148875,
7733181336,
7733123920,
7734249770,
7733814148,
7733126624,
7734233259,
7733813218,
7734690061,
7733147247,
7733109058,
7733168737,
7733820977,
7733857776,
7734689877,
7734232015,
7733131543,
7733826016,
7734039741,
7733853690,
7734230995,
7733146243,
7733020995,
7733023121,
7733121993,
7733536758,
7734408766,
5030032376,
5003021640,
5003021368,
5030032182,
5003096290,
5003021495,
5074019220,
5074019252,
5074045703,
7719872211,
5051005503,
5051005670,
5051005623,
7723187113,
7701905508,
7701377657,
7701113894,
7710929858,
7703021039,
7709330442,
7709833051,
7710961153,
7709944805,
7703379868,
7704222926,
7704853583,
7703112310,
7703363890,
7701032028,
7704178709,
7705871955,
7709344237,
7708155205,
7702764296,
7701375995,
7709157090,
7708071876,
7707290995,
7703261626,
7704222919,
7704118139,
7702058364,
7707094937,
7701384943,
7704231631,
7731180591,
7706807720,
7713795923,
7703808740,
7703611221,
7702797527,
7703776689,
7705552557,
7702833359,
7703738796,
7708057984,
7701406844,
7709387946,
7705480398,
7705052410,
7704040242,
7713757043,
7704118121,
7709439048,
7705041520,
7709366858,
7708044657,
7708235066,
7705513678,
7705513734,
7709438340,
7701050524,
7737027335,
7724916062,
7724732065,
7726644294,
7724189385,
7726705564,
7726008644,
7726073957,
7737078033,
7726032196,
7725056395,
7725766797,
7725144330,
7726737848,
7724857843,
7726643607,
7725687263,
7724224992,
7725034137,
7724731858,
7724731696,
7726643910,
7726643460,
7724294781,
7725684400,
7724184475,
7726076612,
7724240899,
7724851400,
7737124402,
7726510766,
7724304045,
7725259204,
7726711536,
7724848380,
7737078410,
7724191296,
7724942834,
7725143915,
7726726998,
7726180596,
7726269678,
7726287412,
7724886210,
7726308486,
7724191176,
7724947818,
7724912276,
7724204756,
7737119057,
7737119096,
7737106756,
7724914410,
7724733647,
7724189378,
7724030771,
7724917860,
7724859424,
7725247640,
7736000370,
7737057354,
7725247752,
7724035635,
7725281915,
7725031305,
7725247657,
7737132026,
7724295383,
7737014368,
7737066359,
7726316374,
7726548174,
7723920242,
7721692544,
7722306978,
7721805879,
7723918412,
7722069597,
7723324289,
7723186335,
7721816905,
7722266186,
7722006029,
7722820326,
7723169690,
7721219881,
7721151200,
7721816006,
7721267765,
7723305800,
7723327297,
7723927537,
7721809746,
7721146930,
7721179357,
7723329921,
7723881890,
7703742922,
7721854629,
7721749335,
7723882999,
7721805149,
7721804748,
7723929012,
7723398058,
7723320252,
7723881258,
7721237506,
7723864492,
7721223373,
7721817899,
7723138571,
7723196358,
7723162479,
7723163200,
7721227770,
7723103106,
7723374949,
7723356160,
7721222323,
7736210850,
7727233787,
7736212529,
7727190406,
7736210899,
7728887366,
7728007477,
7728900803,
7728231302,
7727844458,
7736614860,
7736682034,
7728261353,
7728237583,
7728231581,
7728900440,
7736133845,
7728229720,
7727123537,
7727190597,
7736132129,
7727209223,
7727203790,
7727190727,
7727633880,
7727801020,
7728013054,
7728007484,
7736076587,
7728229430,
7736004960,
7727190251,
7727063221,
7728232708,
7727844401,
7728860815,
7728122631,
7728849434,
7736221097,
7727763713,
7727801856,
7736228173,
7727190607,
7727809855,
7727213822,
7727236153,
7728006064,
7728291206,
7727824998,
7727506272,
7727506385,
7727670699,
7736668110,
7728858365,
7727840510,
7727844923,
7727842676,
7727177370,
7728873109,
7728173731,
7728261681,
7727190396,
7727047597,
7727824902,
7727028555,
7727846487,
7736050780,
7728246980,
7728021866,
7727847191,
7736110982,
7727270740,
7725539709,
7725618950,
7719210793,
7726317748
]

In [40]:
a = res[(res.inn.isin(inns)) & (res.job_may.isin(ped))]['stv_may'].sum()

In [41]:
b = res_occup[(res_occup.inn.isin(inns)) & (res_occup.job_106.isin(ped))]['good_stv'].sum()

In [42]:
a - b

-43428.95840976658

In [38]:
res_occup['good_stv'] = res_occup.apply(lambda row: to_double(row, 'stv'), axis=1)